In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sys
import h5py
sys.path.insert(0, '..')
from models import PointNet, DGCNNSegBackbone
from datasets import PointCloudNormalize, ABCDataset
from torch.utils.data import Dataset, DataLoader
from utils.training_routines import RunningMetrics
from tqdm import tqdm


## Parameters

In [2]:
device = 'cuda:0'
exp_id = 'z52pya7i'
n_epochs = 50
lr = 5e-4
weight_decay = 1e-5

In [3]:
class Regressor(nn.Module):
    def __init__(self, backbone, finetune_head=False):
        super().__init__()
        self.backbone = backbone
        if finetune_head:
            for p in self.backbone.parameters():
                p.requires_grad = False
                
        self.head = nn.Sequential(
            nn.Conv1d(self.backbone.n_output_point, 512, 1),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Conv1d(512, 256, 1),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Conv1d(256, 1, 1)
        )
        
    def forward(self, x):
        features = self.backbone.forward_features(x)
        return self.head(features).squeeze(1)
    
def compute_loss(gt, pred):
    log_gt = torch.log1p(gt)
    
    return (pred - log_gt).pow(2).mean()

def compute_mse(gt, pred):
    pred = torch.expm1(pred)
    return (pred - gt).pow(2).mean()

@torch.no_grad()
def validate(model, loader):
    model.eval()
    preds = []
    gts = []
    bar = tqdm(loader, desc='val')
    metrics = RunningMetrics()
    
    for x, gt in bar:
        pred = model(x.to(device))
        preds.append(pred.cpu())
        gts.append(gt)
        metrics.step({'loss': compute_loss(gt, pred.cpu())})
        bar.set_postfix(metrics.report())
    
    preds = torch.cat(preds, dim=0)
    gts = torch.cat(gts, dim=0)
    loss = compute_mse(gts, preds).item()
    print(metrics.report())
    
    print('val loss', loss)
    return loss

def train(model, train_loader, test_loader, optimizer, scheduler, n_epochs, val_every=1):
    val_loss_list = []
    for epoch in range(1, n_epochs + 1):
        bar = tqdm(train_loader)
        model.train()
        metrics = RunningMetrics()
        
        for x, gt in bar:
            optimizer.zero_grad()
            pred = model(x.to(device))
            loss = compute_loss(gt.to(device), pred) + 0.001 * model.backbone.reg
            loss.backward()
            optimizer.step()
            metrics.step({'loss': loss})
            report = metrics.report()
            report.update({'epoch': epoch})
            bar.set_postfix(report)
            
            scheduler.step()
        
        if epoch % val_every == 0:
            val_loss = validate(model, test_loader)
            val_loss_list.append(val_loss)
            
    return val_loss_list
            
def get_model(exp_id, n_epochs, finetune_head, lr, weight_decay):
    model = PointNet()
    # model = DGCNNSegBackbone()
    if exp_id is not None:
        checkpoint_path = f'../weights/simclr_run_{exp_id}_ckp_150.pt'
        state = torch.load(checkpoint_path, map_location='cpu')['model']
        model.load_state_dict(state)
        
    model = Regressor(model, finetune_head).to(device)
    optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad,
                                        model.parameters()),
                                 lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs * len(train_loader), eta_min=0)
    return model, optimizer, scheduler

In [4]:
dataset_path = '../../datasets/hdfs/train_0.hdf5'

## Simple training

### Only head

In [5]:
train_ds = ABCDataset(dataset_path,
                      'train',
                      'distances',
                      transform=PointCloudNormalize('box'))
test_ds = ABCDataset(dataset_path,
                     'test',
                     'distances',
                     transform=PointCloudNormalize('box'))

train_loader = DataLoader(train_ds, shuffle=True, batch_size=50)
test_loader = DataLoader(test_ds, shuffle=False, batch_size=32)

In [ ]:
run_results = []

for _ in range(3):
    finetune_head = True
    exp_id = None # 'z52pya7i'
    model, optimizer, scheduler = get_model(exp_id, n_epochs, finetune_head, lr, weight_decay)
    val_loss1 = train(model, train_loader, test_loader, optimizer, scheduler, n_epochs)
    run_results.append(val_loss1[-1])

val: 100%|██████████| 154/154 [00:15<00:00,  9.74it/s, loss=0.0429]


{'loss': 0.04290205746984874}
val loss 0.1440052084294357


val: 100%|██████████| 154/154 [00:15<00:00,  9.89it/s, loss=0.0285]


{'loss': 0.028514715696312476}
val loss 0.07573836657483042


val: 100%|██████████| 154/154 [00:15<00:00,  9.69it/s, loss=0.0241]


...

val: 100%|██████████| 154/154 [00:15<00:00,  9.98it/s, loss=0.016] 


{'loss': 0.01596703342613678}
val loss 0.03597822785476616


val: 100%|██████████| 154/154 [00:15<00:00,  9.77it/s, loss=0.0162]


{'loss': 0.0162437491829253}
val loss 0.03660234335686549


 66%|██████▌   | 151/230 [00:34<00:17,  4.42it/s, loss=0.0251, epoch=47]

In [12]:
np.mean(run_results)

0.009279140784743602

In [ ]:
val_loss1

In [ ]:
finetune_head = True
exp_id = None
model, optimizer, scheduler = get_model(exp_id, n_epochs, finetune_head, lr, weight_decay)
val_loss2 = train(model, train_loader, test_loader, optimizer, scheduler, n_epochs)

In [ ]:
val_loss2

### Finetuning

In [ ]:
finetune_head = False
exp_id = 'z52pya7i'
model, optimizer, scheduler = get_model(exp_id, n_epochs, finetune_head, lr, weight_decay)
val_loss3 = train(model, train_loader, test_loader, optimizer, scheduler, n_epochs)

In [ ]:
val_loss3

### From scratch

In [ ]:
finetune_head = False
exp_id = None
model, optimizer, scheduler = get_model(exp_id, n_epochs, finetune_head, lr, weight_decay)
val_loss4 = train(model, train_loader, test_loader, optimizer, scheduler, 50)

In [ ]:
val_loss4

## Semisupervised

In [ ]:
test_ds = ABCDataset(dataset_path,
                     'test',
                     'distances',
                     transform=PointCloudNormalize('box'))

test_loader = DataLoader(test_ds, shuffle=False, batch_size=32)

In [ ]:
finetune_head = False
exp_id = 'z52pya7i'
n_epochs = 200
run_results = []

for i, seed in enumerate([24234, 23214, 64645]):
    train_ds = ABCDataset(dataset_path,
                          'train',
                          'distances',
                          transform=PointCloudNormalize('box'), sample_frac=0.01, seed=seed)

    train_loader = DataLoader(train_ds, shuffle=True, batch_size=50)
    model, optimizer, scheduler = get_model(exp_id, n_epochs, finetune_head, lr, weight_decay)
    val_loss4 = train(model, train_loader, test_loader, optimizer, scheduler, n_epochs, val_every=50)
    print(f'Run {i}: {val_loss4[-1]}')
    run_results.append(val_loss4[-1])
    
np.mean(run_results), np.std(run_results, ddof=1)

In [ ]:
np.mean(run_results), np.std(run_results, ddof=1)

In [ ]:
finetune_head = False
exp_id = None
n_epochs = 200
run_results = []

for i, seed in enumerate([24234, 23214, 64645]):
    train_ds = ABCDataset(dataset_path,
                          'train',
                          'distances',
                          transform=PointCloudNormalize('box'), sample_frac=0.01, seed=seed)

    train_loader = DataLoader(train_ds, shuffle=True, batch_size=50)

    model, optimizer, scheduler = get_model(exp_id, n_epochs, finetune_head, lr, weight_decay)
    val_loss4 = train(model, train_loader, test_loader, optimizer, scheduler, n_epochs, val_every=50)
    print(f'Run {i}: {val_loss4[-1]}')
    run_results.append(val_loss4[-1])

In [ ]:
np.mean(run_results), np.std(run_results, ddof=1)

In [ ]:
finetune_head = False
exp_id = 'z52pya7i'
n_epochs = 200
run_results = []

for i, seed in enumerate([24234, 23214, 64645]):
    train_ds = ABCDataset(dataset_path,
                          'train',
                          'distances',
                          transform=PointCloudNormalize('box'), sample_frac=0.05, seed=seed)

    train_loader = DataLoader(train_ds, shuffle=True, batch_size=50)

    model, optimizer, scheduler = get_model(exp_id, n_epochs, finetune_head, lr, weight_decay)
    val_loss4 = train(model, train_loader, test_loader, optimizer, scheduler, n_epochs, val_every=50)
    print(f'Run {i}: {val_loss4[-1]}')
    run_results.append(val_loss4[-1])

In [ ]:
np.mean(run_results), np.std(run_results, ddof=1)

In [ ]:
finetune_head = False
exp_id = None
n_epochs = 200
run_results = []

for i, seed in enumerate([24234, 23214, 64645]):
    train_ds = ABCDataset(dataset_path,
                          'train',
                          'distances',
                          transform=PointCloudNormalize('box'), sample_frac=0.05, seed=seed)

    train_loader = DataLoader(train_ds, shuffle=True, batch_size=50)

    model, optimizer, scheduler = get_model(exp_id, n_epochs, finetune_head, lr, weight_decay)
    val_loss4 = train(model, train_loader, test_loader, optimizer, scheduler, n_epochs, val_every=50)
    print(f'Run {i}: {val_loss4[-1]}')
    run_results.append(val_loss4[-1])

In [ ]:
np.mean(run_results), np.std(run_results, ddof=1)